In [1]:
import requests
import json

The PubMed API has two main endpoints:

- ESearch — search for papers, get IDs
- EFetch — use IDs to get the actual conten

### Search: sleep improves cognitive performance


In [2]:
response = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={
        "db": "pubmed",
        "term": "air quality",
        "retmode": "json",
        "retmax": 100
    }
)

result = response.json()
print(json.dumps(result, indent=2))

{
  "header": {
    "type": "esearch",
    "version": "0.3"
  },
  "esearchresult": {
    "count": "143335",
    "retmax": "100",
    "retstart": "0",
    "idlist": [
      "41521142",
      "41521015",
      "41520990",
      "41520753",
      "41520550",
      "41520496",
      "41520438",
      "41520382",
      "41520003",
      "41519986",
      "41519885",
      "41519348",
      "41519291",
      "41519288",
      "41519284",
      "41519282",
      "41518982",
      "41518919",
      "41518777",
      "41518628",
      "41518522",
      "41518475",
      "41518427",
      "41518347",
      "41518040",
      "41517131",
      "41517106",
      "41516928",
      "41516765",
      "41516757",
      "41516628",
      "41516568",
      "41516520",
      "41516442",
      "41516222",
      "41516208",
      "41515969",
      "41515842",
      "41515811",
      "41515788",
      "41514798",
      "41514771",
      "41514710",
      "41514520",
      "41514494",
      "41514397",
     

In [3]:
ids = result["esearchresult"]["idlist"]  # your list of IDs
id_string = ",".join(ids)  # turns list into "123,456,789,..."


response2 = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
    params={
        "db": "pubmed",
        "id": id_string,
        "retmode": "xml"
    }
)

print(response2.text)

<?xml version="1.0" ?>
<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2025//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_250101.dtd">
<PubmedArticleSet>
<PubmedArticle><MedlineCitation Status="MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">41521142</PMID><DateCompleted><Year>2026</Year><Month>01</Month><Day>11</Day></DateCompleted><DateRevised><Year>2026</Year><Month>01</Month><Day>11</Day></DateRevised><Article PubModel="Print-Electronic"><Journal><ISSN IssnType="Electronic">1473-2300</ISSN><JournalIssue CitedMedium="Internet"><Volume>54</Volume><Issue>1</Issue><PubDate><Year>2026</Year><Month>Jan</Month></PubDate></JournalIssue><Title>The Journal of international medical research</Title><ISOAbbreviation>J Int Med Res</ISOAbbreviation></Journal><ArticleTitle>Associations of PM<sub>2.5</sub> and its major chemical components with cognitive function: A nationwide prospective cohort study among middle-aged and older adults in China.</A

In [4]:
import xml.etree.ElementTree as ET

root = ET.fromstring(response2.text)

papers = []
for article in root.findall(".//PubmedArticle"):
    # Find the title
    title = article.find(".//ArticleTitle").text
    # Find all abstract sections and combine them
    abstract_parts = article.findall(".//AbstractText")
    abstract = " ".join([part.text for part in abstract_parts if part.text])
    papers.append({"title": title, "abstract": abstract})

print(len(papers))
papers[:3]

100


[{'title': 'Associations of PM',
  'abstract': 'ObjectiveThis study aimed to investigate the associations between fine particulate matter and its major chemical components and cognitive function among middle-aged and older adults in China.MethodsWe conducted a nationwide prospective cohort study using data from the China Health and Retirement Longitudinal Study (2011-2018). Cognitive function was repeatedly assessed through standardized tests of memory and mental status. Annual average concentrations of fine particulate matter and its five major components (sulfate, nitrate, ammonium, black carbon, and organic matter) were estimated at the city level. Fixed-effects models and restricted cubic spline analyses were used to evaluate associations, and random forest models were used to rank the relative importance of components.ResultsHigher exposure to fine particulate matter and several of its major components was significantly associated with lower cognitive scores. Among these component

In [5]:
print("TITLE:", title)
print("\nABSTRACT:", abstract[:500])

TITLE: Environmental determinants of cerebral haemorrhage in older adults: behavioural pathways and population health implications.

ABSTRACT: Intracerebral haemorrhage (ICH) is a rapidly fatal cerebrovascular catastrophe that claims a disproportionate share of stroke deaths among older adults despite decades of progress in acute care. Emerging research now implicates a constellation of non-biomedical contextual stressors ambient fine particulate matter, traffic-derived gases and noise, thermal volatility, and bio-accumulative heavy metals, themselves patterned by social determinants of health and political-commercial decision-making a
